# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

# Описание данных

**Данные геологоразведки трёх регионов разбиты на три файла.**

**Описание признаков**
- **id** — уникальный идентификатор скважины;
- **f0, f1, f2** — три признака точек (неважно, что они означают, но сами признаки значимы);
- **product** — объём запасов в скважине (тыс. баррелей).

# План работы:

- **Загрузка и подготовка данных.**

- **Обучение и проверки модели для каждого региона:**
     - Разбивка данных на обучающую и валидационную выборку;
     - Обучение модели и предсказание на валидационной выборке;
     - Расчет среднего запаса предсказанного сырья и RMSE модели;
     - Анализ результатов;
     
- **Подготовка к расчету прибыли:**
    - Расчет достаточного объема сырья для безубыточной разработки новой скважины. Сравнение полученных результатов со средним запасом в каждом регионе;
    - Анализ результатов;
   
- **Расчет прибыли:**
    - Выбор скважин с максимальными значениями предсказаний;
    - Суммирование целевого значения объема сырья, соответствующее этим предсказаниям;
    - Непосредственный расчте прибыли для полученного объема сырья;
    
- **Расчет рисков и прибыли для каждого региона:**
    - Применение техники Bootstrap с 1000 выборок, для поиска распределение прибыли;
    - Расчет средней прибыли, 95%-ого доверительного интервала и риска убытков;
    - Итоговый вывод;

# Импорт библиотек:

In [2]:

import warnings; warnings.filterwarnings("ignore", category=Warning)

import pandas as pd
import numpy as np
from scipy import stats as st

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

pd.options.mode.chained_assignment = None

# 1. Загрузка и подготовка данных

In [3]:
df_region1 = pd.read_csv('/datasets/geo_data_0.csv')
df_region2 = pd.read_csv('/datasets/geo_data_1.csv') 
df_region3 = pd.read_csv('/datasets/geo_data_2.csv') 

df_list = [df_region1, df_region2, df_region3]
name_df_list = ['"Регион №1"', '"Регион №2"', '"Регион №3"']

Данные загружены в переменные, теперь взглянем на их шапки, общую информацию, и выведем на экран конечное количество строк, пропущенных значений, и дубликатов

In [4]:
for i in range(len(df_list)):
    print('\033[1m' + 'Наименование датафрейма - ' + name_df_list[i], ':' + '\033[0m')
    print('\033[1m' + 'Шапка датафрейма:' + '\033[0m')
    display(df_list[i].head(5))
    print('')
    print('\033[1m' + 'Общая информация о датафрейме' + '\033[0m')
    display(df_list[i].info())
    print('\033[1m' + 'Информация о размере, пропусках и дубликатах в датафрейме' + '\033[0m')
    print('Количество строк -', len(df_list[i]))
    print('Количество пропущенных значений -', 
          df_list[1][df_list[1].isna() == True].sum().sum())
    print('Количество дубликатов -', df_list[i].duplicated().sum())
    print('')
    print('')

Наименование датафрейма - "Регион №1" :
Шапка датафрейма:


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647



Общая информация о датафрейме
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

Информация о размере, пропусках и дубликатах в датафрейме
Количество строк - 100000
Количество пропущенных значений - 0.0
Количество дубликатов - 0


Наименование датафрейма - "Регион №2" :
Шапка датафрейма:


...

Информация о размере, пропусках и дубликатах в датафрейме
Количество строк - 100000
Количество пропущенных значений - 0.0
Количество дубликатов - 0


Наименование датафрейма - "Регион №3" :
Шапка датафрейма:


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746



Общая информация о датафрейме
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

Информация о размере, пропусках и дубликатах в датафрейме
Количество строк - 100000
Количество пропущенных значений - 0.0
Количество дубликатов - 0




Наблюдаем, что в трех датафреймах одинаковое количество строк (100 тыс. значений), столбцов, отсутствуют пропущенные значения и дубликаты. Типы данных также находятся в полном порядке. Для окончательной подготовки данных к анализу нам необходимо лишь избавиться от столбца id, так как в дальнейшем он нам не понадобится.

In [5]:
for i in range(len(df_list)):
    df_list[i] = df_list[i].drop(['id'], axis = 1)

Столбец 'id' успешно удален, данные готовы к анализу. Переходим к следующему этапу.

# 2. Обучение и проверки модели для каждого региона:

В первую очередь, подготовим списки с признаками и целевыми признаками по каждому региону.

In [6]:
x_par_list = []
y_tar_list = []
for i in range(len(df_list)):
    x_par_list.append(df_list[i].drop(['product'], axis = 1))
    y_tar_list.append(df_list[i]['product'])

Данные подготовлены для разбивки, теперь напишем функцию, которая на входе получит пустые списки с признаками и целевыми признаками для тренировочной и валидационной выборки по каждому региону. Внутри функция выполнит масштабирование числовых признаков (т.е. всех), разобъет датафреймы на выборки, и добавит их в пустые списке. На выходе функция выдаст четыре списка , внутри которых будут находится тренировочные и валидационные выборки по трем регионам.

In [7]:
x_train_list = []
x_valid_list = []
y_train_list = []
y_valid_list = []
numeric = ['f0', 'f1', 'f2']
def train_valid_splitter(x_train_list, x_valid_list, y_train_list, y_valid_list):
    
    for i in range(len(x_par_list)):
                
        features_train, features_valid, target_train, target_valid = train_test_split(
            x_par_list[i], 
            y_tar_list[i], 
            test_size = 0.25)
        
        scaler = StandardScaler()
        scaler.fit(features_train)
        features_train = scaler.transform(features_train)
        features_valid = scaler.transform(features_valid)
        
        print('\033[1m' + 'Наименование исходного датафрейма - ' + name_df_list[i] + '\033[0m')
        print('Размер обучающей выборки: ', features_train.shape)
        print('Размер валидационной выборки: ', features_valid.shape)
        print('')
        x_train_list.append(features_train)
        x_valid_list.append(features_valid)
        y_train_list.append(target_train)
        y_valid_list.append(target_valid)
        
    return x_train_list, x_valid_list, y_train_list, y_valid_list

Функция подготовлена, проверим её работоспособность.

In [8]:
x_train_list, x_valid_list, y_train_list, y_valid_list = train_valid_splitter(
    x_train_list, x_valid_list, y_train_list, y_valid_list)

Наименование исходного датафрейма - "Регион №1"
Размер обучающей выборки:  (75000, 3)
Размер валидационной выборки:  (25000, 3)

Наименование исходного датафрейма - "Регион №2"
Размер обучающей выборки:  (75000, 3)
Размер валидационной выборки:  (25000, 3)

Наименование исходного датафрейма - "Регион №3"
Размер обучающей выборки:  (75000, 3)
Размер валидационной выборки:  (25000, 3)



Функция применена успешно, получили по три набора выборок. 

Теперь переходим непосредственно к обучению модели линейной регрессии, для этого будем использовать автоподбор гиперпараметров с помощью GridSearchCV. Также сразу же попробуем предсказать нужные нам значения на валидационной выборке, добавим их в пустой список, и рассчитаем метрики для оценки адекватности нашей модели - средний запас предсказанного сырья, RMSE, MAE и коэффициент детерминации. 

Полученные результаты выведем на экран.

In [9]:
parametrs = {'fit_intercept':('True', 'False'), 
             'normalize':('True', 'False'), 
             'copy_X':('True', 'False')}
results_list = []
mean_results_list = []
for i in range(len(x_train_list)):
    
    model_name = LinearRegression()
    
    grid = GridSearchCV(model_name, parametrs, cv = 5)
    grid.fit(x_train_list[i], y_train_list[i])

    model = LinearRegression(**grid.best_params_)
    
    model.fit(x_train_list[i], y_train_list[i])
    
    model_train_predictions = model.predict(x_train_list[i])
    model_valid_predictions = model.predict(x_valid_list[i])
    
    results_list.append(model_valid_predictions)
    mean_results_list.append(round(sum(results_list[i]) / len(results_list[i]), 3))
    rmse = mean_squared_error(y_valid_list[i], model_valid_predictions) ** 0.5
    mae = mean_absolute_error(y_valid_list[i], model_valid_predictions)
    r2 = r2_score(y_valid_list[i], model_valid_predictions)
    
    print('')
    print('\033[1m' + 'Датафрейм -', name_df_list[i] + '\033[0m')
    print('Средний запас предсказанного сырья в датафрейме составил -', mean_results_list[i])
    print('RMSE -', round(rmse, 3))
    print('MAE -', round(mae, 3))
    print('Коэффициент детерминации -', round(r2, 5))


Датафрейм - "Регион №1"
Средний запас предсказанного сырья в датафрейме составил - 92.448
RMSE - 37.68
MAE - 31.04
Коэффициент детерминации - 0.27321

Датафрейм - "Регион №2"
Средний запас предсказанного сырья в датафрейме составил - 68.712
RMSE - 0.893
MAE - 0.72
Коэффициент детерминации - 0.99962

Датафрейм - "Регион №3"
Средний запас предсказанного сырья в датафрейме составил - 95.055
RMSE - 39.954
MAE - 32.724
Коэффициент детерминации - 0.20391


Исходя из полученных результатов, наблюдаем что наибольшие средние предсказанные запасы сырья находятся в регионах №1 и №3, однако в них наиболее высоки значения RMSE и MAE и низкий коэффициент детерминации - что в свою очередь, означает что связи между признаками и целевым признаком недостаточно существенны. В регионе №2 наиболее низкий средний предсказанный запас сырья, однако практически максимальный коэффициент детерминации (0.999), что указывает на полную взаимосвязь между признаками и целевым признаком, и наиболее низкие показатели RMSE и MAE.

# 3. Подготовка к расчёту прибыли

**Для начала внесем необходимые для нас константы:**
 - BUDGET - бюджет на разработку скважин в регионе — 10 млрд рублей.
 - OILWELLS - 200 лучших месторождений при разведке региона
 - BUDGET_PER_OILWELL - бюджет на разработку одного месторождения
 - PROFIT_PER_PRODUCT - прибыль с одной единицы продукта - 450 тыс. руб.
 - MAX_RISK_PROB - вероятность убытков, не более 2.5%
 - MLRD - числовое значение миллиадра для удобства расчетов.

In [10]:
BUDGET = 10**10
OILWELLS = 200
BUDGET_PER_OILWELL = BUDGET / OILWELLS
PROFIT_PER_PRODUCT = 450000
MAX_RISK_PROB = 0.025
MLRD = 1000000000

Теперь рассчитаем объем сырья для безубыточности разработки новой скважины, и выведем его на экран. 
После этого также выведем на экран средний истинный объем сырья, средний предполагаемый объем сырья, абсолютные отклонения между средним истинным объемом сырья и объемом сырья для безубыточности, и между средним предполагаемым объемом сырья и объемом сырья для безубыточности по каждому региону.

In [11]:
BREAKEVEN_POINT = round(BUDGET_PER_OILWELL / PROFIT_PER_PRODUCT, 2)
print('Объем сырья для безубыточности разработки новой скважины =', BREAKEVEN_POINT)
for i in range(len(mean_results_list)):
    print("")
    print("")
    print('\033[1m' + name_df_list[i] + '\033[0m')
    print('Средний истинный объем сырья =', round(y_tar_list[i].mean(), 3))
    print('Абсолютное отклонение =', round(y_tar_list[i].mean() - BREAKEVEN_POINT, 3))
    print('')
    print('Средний предполагаемый объем сырья =', round(mean_results_list[i], 3))
    print('Абсолютное отклонение =', round(mean_results_list[i] - BREAKEVEN_POINT, 3))

Объем сырья для безубыточности разработки новой скважины = 111.11


"Регион №1"
Средний истинный объем сырья = 92.5
Абсолютное отклонение = -18.61

Средний предполагаемый объем сырья = 92.448
Абсолютное отклонение = -18.662


"Регион №2"
Средний истинный объем сырья = 68.825
Абсолютное отклонение = -42.285

Средний предполагаемый объем сырья = 68.712
Абсолютное отклонение = -42.398


"Регион №3"
Средний истинный объем сырья = 95.0
Абсолютное отклонение = -16.11

Средний предполагаемый объем сырья = 95.055
Абсолютное отклонение = -16.055


Видим что по средним значениям ни один регион не подходит для разработки, отклонения наблюдаются в разбросе от 14 до 40%. Однако, в дальнейшем, мы будем исследовать лишь 200 наиболее прибыльных месторождений в каждом регионе.

Подготовка к расчету прибыли завершена, переходим непосредственно к самому расчету прибыли и рисков.

# 4. Расчёт прибыли и рисков 

Для расчета прибыли и рисков, напишем функцию которая на входе будет получать переменную purpose для уточнения использования функции, и непосредственный датафрейм. На выходе функция либо выведет на экран итоговую прибыль по 200 наиболее прибыльным месторождениям в регионе, либо подготовит рассчитаную прибыль для использования её в методе Bootstrap.

In [12]:
def profit_calc(purpose, data):
    if purpose == 'Расчет прибыли':
        for i in range(len(df_list)):
            top200 = pd.DataFrame(data[i], columns = ['predicted_results']).reset_index(drop = True)
            top200 = top200.sort_values(by = 'predicted_results', ascending = False)[:OILWELLS]
            profit = ((top200.sum() * PROFIT_PER_PRODUCT) - BUDGET) / MLRD
            print('')
            print('\033[1m' + 'Наименование региона -', name_df_list[i]+ '\033[0m')
            print('Итоговая прибыль -', round(profit[0], 3), 'в млрд. руб.')
    elif purpose == 'Bootstrap':
        profit = ((data.sum() * PROFIT_PER_PRODUCT) - BUDGET) / MLRD
        return profit

Функция готова к применению, используем её.

In [13]:
profit_calc('Расчет прибыли', results_list)


Наименование региона - "Регион №1"
Итоговая прибыль - 3.934 в млрд. руб.

Наименование региона - "Регион №2"
Итоговая прибыль - 2.485 в млрд. руб.

Наименование региона - "Регион №3"
Итоговая прибыль - 3.392 в млрд. руб.


Видим, что как и наблюдалось ранее - первый и третий регион наиболее прибыльны. Однако нам уже известны по какой причине они являются прибыльными, теперь же проверим их эффективность с помощью метода Bootstrap. После этого выведем на экран по каждому региону границы 95% доверительного интервала, среднюю прибыль, и риск убытков в %.

In [24]:
state = np.random.RandomState(12345)

for j in range(len(y_valid_list)):
    values = []
    
    subsample = pd.Series(y_valid_list[j]) 
    
    for i in range(1000):
        predicted_top200 = pd.DataFrame(results_list[j], columns = ['product']).reset_index(drop = True)
        predicted_top200 = predicted_top200.sample(n=500, replace=True, random_state=state)
        predicted_top200 = predicted_top200.sort_values(by = 'product', ascending = False)
        
        subsample = pd.Series(y_valid_list[j]).reset_index(drop = True)
        subsample = subsample[predicted_top200.index][:OILWELLS]
        
        values.append(profit_calc('Bootstrap', subsample))
    
    values = pd.Series(values)    
    confidence_interval = st.t.interval(0.95, len(values)-1, values.mean(), values.sem())
    mean = values.mean()
    risk = (values < 0).mean()
    
    print('\033[1m' + 'Наименование региона: ' + name_df_list[j] + '\033[0m')
    print("Нижняя граница доверительного интервала: {:,.4f} млрд. руб.".format(confidence_interval[0]))
    print("Верхняя граница доверительного интервала: {:,.4f} млрд. руб.".format(confidence_interval[1]))
    print("Средняя прибыль: {:,.4f} млрд. руб.".format(mean))
    print("Риск убытков: {:,.2%}.".format(risk))
    print('')

Наименование региона: "Регион №1"
Нижняя граница доверительного интервала: 0.3597 млрд. руб.
Верхняя граница доверительного интервала: 0.3922 млрд. руб.
Средняя прибыль: 0.3760 млрд. руб.
Риск убытков: 7.80%.

Наименование региона: "Регион №2"
Нижняя граница доверительного интервала: 0.4394 млрд. руб.
Верхняя граница доверительного интервала: 0.4652 млрд. руб.
Средняя прибыль: 0.4523 млрд. руб.
Риск убытков: 1.50%.

Наименование региона: "Регион №3"
Нижняя граница доверительного интервала: 0.3983 млрд. руб.
Верхняя граница доверительного интервала: 0.4322 млрд. руб.
Средняя прибыль: 0.4153 млрд. руб.
Риск убытков: 6.00%.



Теперь же мы наблюдаем совершенно иные результаты - регион №2 имеет как наибольшую прибыль, так и наибольший доверительный интервал с наименьшими рисками убытков всего в 0.7%. Регион №1 и регион №3 по размерам прибыли являются оптимальными для разработки, однако риски убытков в этих регионах свыше 5.5%, что не допустимо для наших условий.

# Итоговый вывод:

Предварительно обработав данные, разбив их на обучающие и валидационные выборки, проведя построение модели линейной регрессии на полученных выборках, рассчитаев метрики для полученных моделей и получив доверительные интервалы и среднюю прибыль по каждому региону, можем сделать следующий вывод:

- **Наиболее прибыльным регионом является регион №2**, т.к. в нем наиболее высокая средняя прибыль, максимальный коэффициент детерминации (что свидетельствует о наличии высокой взаимосвязи между признаками и целевым признаком), с наименьшими рисками убытков (0.7%).
- Регионы №1 и №3 не рекомендуются к разработке за счет низкого значения коэффициента детерминации, сниженного объема прибыли и повышенных рисков убытков в 5.5-7%. 